In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import keras
from keras import layers, activations
from os import path
from PIL import Image
import pickle

trainingDataDir = 'o:/temp/pixiv/training/'
targetSize = (224, 224)
targetShape = (224, 224, 3)

In [ ]:
def conv_block(x, filters, kernel_size=3, strides=1, padding='same'):
    x = layers.Conv2D(filters, kernel_size, strides=strides,
                      padding=padding)(x)
    x = layers.Conv2D(filters, kernel_size, strides=strides,
                      padding=padding)(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D()(x)
    return x


def preprocess(x):
    x = layers.RandomZoom(height_factor=0.1, width_factor=0.1)(x)
    x = layers.RandomFlip()(x)
    x = layers.RandomRotation(factor=0.1)(x)
    x = layers.Rescaling(scale=1. / 127.5,
                         offset=-1,
                         input_shape=targetShape)(x)
    return x


appModel = keras.applications.mobilenet_v2.MobileNetV2(include_top=False,
                                       weights='imagenet',
                                       input_shape=targetShape)
appModel.summary()
for layer in appModel.layers:
    layer.trainable = False


inputs = layers.Input(shape=targetShape)
x = preprocess(inputs)
x = appModel(x)
x = layers.GlobalAveragePooling2D()(x)
# x = layers.Flatten()(x)

x = layers.Dropout(0.3)(x)
x = layers.Dense(1024)(x)
x = layers.Activation(activation=activations.leaky_relu)(x)

x = layers.Dropout(0.3)(x)
x = layers.Dense(1024)(x)
x = layers.Activation(activation=activations.leaky_relu)(x)

x = layers.Dense(5, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=x)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()


In [ ]:
def get_data():
    if path.exists('ds.pickle'):
        with open('ds.pickle', 'rb') as f:
            ds = pickle.load(f)
        return ds
    else:
        df = pd.read_csv(path.join(trainingDataDir, 'data.csv'))
        imgs = []
        for i in range(len(df)):
            img = Image.open(path.join(trainingDataDir, df['img'][i]))
            xx = max(img.width, img.height)
            new_img = Image.new('RGB', (xx, xx))
            new_img.paste(img, (int(
                (xx - img.width) / 2), int((xx - img.height) / 2)))
            img = new_img.resize(targetSize, Image.BICUBIC)
            img = np.array(img)
            imgs.append(img)
        x = np.array(imgs)
        # y = np.array(df['score'].values)
        y = []
        for v in df['score'].values:
            print(v)
            row = np.zeros(5)
            row[v - 1] = 1
            y.append(row)
        y = np.array(y)
        with open('ds.pickle', 'wb') as f:
            pickle.dump((x, y), f)
        return x, y


(x, y) = get_data()


In [ ]:
cbs = [
    keras.callbacks.EarlyStopping(monitor='val_accuracy',
                                  patience=10,
                                  verbose=1,
                                  restore_best_weights=True),
]

model.fit(
    x,
    y,
    callbacks=cbs,
    epochs=1000,
    batch_size=32,
    validation_split=0.25,
)


In [ ]:
def predict(img_path):
    img = Image.open(img_path)
    xx = max(img.width, img.height)
    new_img = Image.new('RGB', (xx, xx))
    new_img.paste(img, (int((xx - img.width) / 2), int((xx - img.height) / 2)))
    img = new_img.resize(targetSize, Image.BICUBIC)
    img = np.array(img)
    img = img.reshape((1, 224, 224, 3))
    return model.predict(img)


pred = predict(path.join('o:/temp/pixiv/test/', '96284448_p7.jpg'))
pred_score = np.argmax(pred) + 1
print(pred, pred_score)


In [ ]:
model.save('model.h5')